[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

**Author and Copyright Notice** *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*

# UK Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the EECS JupyterHub as well as in the binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
# import library
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [8]:
# First step: Load initial data from disk
# Reading JSON file from a local file
with open('raw_data.json','r') as f:
    data = json.loads(f.read())

In [4]:
# Second step: Wrangle the data from JSON file into a DataFrame
def wrangle_data(data):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe."""
    # Convert JSON file into a Pandas DataFrame
    # Flatten nested list and dict from JSON object
    df = pd.json_normalize(data, record_path =['data'])
    # Reverse the row and reset index 
    df = df.loc[::-1].reset_index(drop=True)
    # Fill in NAN value with 0.0
    df.fillna(0.0, inplace=True)
    
    return df

# Wrap wrangling code into a function allows to call it again after refreshing the data through 
# the API. 
df = wrangle_data(data) 

In [5]:
# Third step: Download the latest data by accessing the API.
def access_api():
    """ Access the PHE API. Return raw data in the same format as data loaded from the "canned" JSON file. """
    england_only = [
        'areaType=nation',
        'areaName=England'
    ]

    cases_death_and_vaccin = {
        "date": "date",
        "totalCases": "cumCasesByPublishDate",
        "totalDeaths": "cumDeaths28DaysByDeathDate",
        "totalAdmissions": "cumAdmissions"

    }

    api = Cov19API(
        filters=england_only,
        structure=cases_death_and_vaccin,
    )

    data = api.get_json()
    
    return data # return data read from the API

In [6]:
# Forth Step: Set a button to refresh data and redraw the graph
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. 
    apidata = access_api()
    
    # Wrangle the data and overwrite the dataframe for plotting
    global df
    df = wrangle_data(apidata)
    
    # Redraw the graph with lastest data access from API
    refresh_graph()
    
    # Change the status of the button
    apibutton.icon="check"

    
apibutton=wdg.Button(
    description='REFRESH',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='rotate-right'
)

# Register button callback function with the button
apibutton.on_click(api_button_callback)

display(apibutton)

Button(description='REFRESH', icon='rotate-right', style=ButtonStyle(), tooltip='Keep calm and carry on')

In [7]:
# Fifth Step: Display the interactive graph
# Draw a bar graph with xaxis is date and yaxis is col.options
def graph(graphcolumns):
    plt.bar(df['date'], df[graphcolumns])
    plt.xticks(np.arange(50, 1031, 60), rotation=90) # set xaxis tick
    # plt.xlim([1010, 1030]) # test for the refresh button
    plt.show()
    
    
cols = wdg.Dropdown(
    options=['totalCases', 'totalDeaths', 'totalAdmissions'], # options available
    value='totalCases', # initial value
    description='Overview: ',
    disabled=False
)

def refresh_graph():
    """ Change the value of the widget in order to force a redraw of the graph."""
    current=cols.value
    if current==cols.options[0]:
        other=cols.options[1]
    elif current==cols.options[1]:
        other=cols.options[2]
    else:
        other=cols.options[0]
    cols.value=other # forces the redraw
    cols.value=current # now change it back
    
# keep calling graph(graphcolumns=value_of_cols); capture output in widget output    
output = wdg.interactive_output(graph, {'graphcolumns': cols})

display(cols, output)

Dropdown(description='Overview: ', options=('totalCases', 'totalDeaths', 'totalAdmissions'), value='totalCases…

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 